In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
postings = pd.read_csv('job-postings\job_postings.csv')
postings.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,NaN,NaN,NaN,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420
2,3757938019,474443.0,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644
3,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,NaN,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346


In [28]:
filtered_postings = postings[postings['skills_desc'].notnull()]
filtered_postings = filtered_postings['skills_desc']
filtered_postings.head(20)

2       Bachelor's Degree in Mechanical Engineering pr...
10      Education: HS Diploma or equivalent is require...
20      10-12 years of directly related experience in ...
89      Previous retail management experience a plusSt...
199                             Ultrasound Tech - General
318                                     ER Emergency Room
479                               Sterile Processing Tech
507                               Sterile Processing Tech
580                      Licensed Practical Nurse LPN/LVN
699                    IR Tech - Interventional Radiology
739                                     OR Operating Room
1253    We want you to have:Current Esthetician Licens...
1331    \n\nURGENT CARE NURSE PRACTITIONER EDUCATION, ...
1351    Essential Functions:This position is responsib...
1374    What you will love doing: Meet/exceed monthly,...
1436    \nMinimum Job Requirements:\nValid driver's li...
1444    \nRequirements:Provides memorable guest servic...
1448    \nRequ

In [29]:
import re

# Extract sentences from all job descriptions
all_sentences = []
for description in filtered_postings:
    # Split the description into sentences using regex
    sentences = re.split(r'(?<=[.!?]) +', description)
    # Remove whitespace from each sentence and add to the list
    all_sentences.extend([sentence.strip() for sentence in sentences if sentence.strip()])

# Define the file path
file_path = "sentences_from_job_descriptions.txt"

# Write sentences to the text file
with open(file_path, "w") as file:
    for sentence in all_sentences:
        file.write(sentence + "\n")


In [30]:
import spacy
import pandas as pd

# Load the spaCy English model
nlp = spacy.load("en_core_web_lg")

# Define a function to extract requirements from skills_desc
def extract_requirements(text):
    # Process the text using spaCy
    doc = nlp(text)
    
    # Extract sentences
    requirements = [sent.text.strip() for sent in doc.sents]
    
    return requirements

# Example filtered_postings
filtered_postings = postings[postings['skills_desc'].notnull()]['skills_desc']

# Apply the extract_requirements function to each element of filtered_postings
requirements_array = filtered_postings.apply(extract_requirements)

# Convert the resulting Series of lists into a single array
all_requirements = []
for requirements_list in requirements_array:
    all_requirements.extend(requirements_list)

# Convert the list into a numpy array if needed
import numpy as np
all_requirements_array = np.array(all_requirements)


In [31]:
for i, requirement in enumerate(all_requirements, start=1):
    if requirement.startswith("Requirement"):
        requirement = requirement.split(":")[1].strip()
    print(f"Requirement {i}: {requirement}")

# Print the total count of requirements
total_count = len(all_requirements)
print(f"\nTotal count of requirements: {total_count}")


Requirement 1: Bachelor's Degree in Mechanical Engineering preferred or equivalent work experience.10 years working in an Industrial Equipment Construction Environment.
Requirement 2: Experience in Conveyor/Material Handling Automotive Market.
Requirement 3: Experience in Fixtures/Enclosures/Ductwork Industrial Market Outstanding communication and organizational skills.
Requirement 4: Ability to work in a fast-paced environment.
Requirement 5: Knowledge of structural steel elements, material handling and production systems.
Requirement 6: Knowledge of enclosure, ductwork, and fixture design.
Requirement 7: State registered Professional Engineer, preferred.
Requirement 8: Attention to detail with emphasis on excellent customer service.
Requirement 9: Fosters and supports an environment of strong teamwork.
Requirement 10: Strong commitment to safety, departmental goals, and objectives.
Requirement 11: Ability to prioritize work and meet deadlines.
Requirement 12: Ability to work independ

In [32]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import random
from spacy.training import Example

# Preparing training data
TRAIN_DATA = []

# Prepare training data from extracted requirements
for requirement in all_requirements:
    # Extract the start and end indices for the entire requirement
    start_idx = 0
    end_idx = len(requirement)
    
    # Append the requirement and its indices to the training data
    TRAIN_DATA.append((requirement, {"entities": [(start_idx, end_idx, "REQUIREMENT")]}))

# Shuffle the training data
random.shuffle(TRAIN_DATA)

# Update the model
for text, annotations in TRAIN_DATA:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    nlp.update([example], drop=0.5, losses={})

ValueError: Shape mismatch for blis.gemm: (0, 0), (672, 288)

In [ ]:
# Select the 10th job posting description
description = "Are you a skilled and innovative software engineer seeking new challenges in a dynamic environment? We are looking for a talented individual to join our team and contribute to the development of cutting-edge software solutions. As a software engineer, you will be responsible for designing, developing, and maintaining high-quality software applications that meet the needs of our clients and users. Responsibilities include collaborating with cross-functional teams to gather and analyze requirements, designing and implementing scalable and efficient software solutions, writing clean and maintainable code, conducting thorough testing to ensure software quality and reliability, troubleshooting and debugging issues in existing software applications, and staying up-to-date with the latest technologies and best practices in software development. Requirements include a bachelor's degree in Computer Science, Engineering, or related field, proficiency in one or more programming languages such as Python, Java, C++, or JavaScript, strong understanding of software engineering principles and methodologies, experience with web development frameworks (e.g., Django, Flask, Spring), familiarity with database systems (e.g., SQL, NoSQL), excellent problem-solving and analytical skills, and effective communication and teamwork abilities. If you are passionate about technology and eager to make a difference in the world of software development, we want to hear from you! Apply now to join our team and take your career to the next level."

# Process the description using the trained spaCy model
doc = nlp(description)

# Extract requirements from the entities recognized by the model
requirements_10th = [ent.text for ent in doc.ents if ent.label_ == 'REQUIREMENT']

# Print the formatted requirements
print("Requirements:")
for i, requirement in enumerate(requirements_10th, start=1):
    print(f"{i}. {requirement}")

Requirements:
1. Are you a skilled and innovative software engineer seeking new challenges in a dynamic environment?
2. We are looking for a talented individual to join our team and contribute to the development of cutting-edge software solutions.
3. As a software engineer, you will be responsible for designing, developing, and maintaining high-quality software applications that meet the needs of our clients and users.
4. Responsibilities include collaborating with cross-functional teams to gather and analyze requirements, designing and implementing scalable and efficient software solutions, writing clean and maintainable code, conducting thorough testing to ensure software quality and reliability, troubleshooting and debugging issues in existing software applications, and staying up-to-date with the latest technologies and best practices in software development.
5. Requirements include a bachelor's degree in Computer Science, Engineering, or related field, proficiency in one or more p